In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [5]:
# Set dataset path
data_path = "/content/drive/MyDrive/DATASET/IMBD WIKI"

In [6]:
def extract_age_from_filename(filename):
    try:
        parts = filename.split("_")
        birth_year = int(parts[2].split("-")[0])  # Extract birth year
        photo_year = int(parts[3].split(".")[0])  # Extract photo year
        return photo_year - birth_year  # Compute age
    except:
        return None

def load_data(data_path, img_size=(224, 224)):
    images = []
    ages = []
    for filename in os.listdir(data_path):
        img_path = os.path.join(data_path, filename)
        if filename.endswith(".jpg"):
            age = extract_age_from_filename(filename)
            if age is not None:
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, img_size)
                    img = img / 255.0  # Normalize
                    images.append(img)
                    ages.append(age)
                except:
                    continue
    return np.array(images), np.array(ages)

In [7]:

# Load images and labels
X, y = load_data(data_path)

In [8]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:

# Load pre-trained VGG16 model (excluding top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:

# Add custom layers for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1)(x)  # Regression output

In [11]:
# Define final model
model = Model(inputs=base_model.input, outputs=x)

In [12]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [13]:
# Save model
model.save("fine_tuned_age_detection_model.h5")

In [14]:
print("Fine-tuned model training complete and saved.")

Fine-tuned model training complete and saved.
